In [1]:
#!/usr/bin/python3
# coding: utf-8
import pandas as pd
import numpy as np
from time import time
import os
import random
import xlrd
import xlwt
from scipy import optimize
import scipy
from tqdm import tqdm_notebook
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, mutual_info_classif
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import manifold
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


########################################################################################################################
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = scipy.sparse.csc_matrix(np.diff(np.eye(L), 2))
    w = np.ones(L)
    for i in range(niter):
        W = scipy.sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = scipy.sparse.linalg.spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z




########################################################################################################################
# position confirm 定位
def wavelength_position(wl, wl_range):

    output = []
    for index in range(len(wl_range)):
        output.append(wl.index(wl_range[index]))

    return output


########################################################################################################################
# 线性拟合
def mean_spectra(x, n):
    r_in, c_in = x.shape
    r_out = int(r_in/n)
    x_out = np.zeros([r_out, c_in])
    for indexi in range(r_out):
        x_out[indexi] = np.mean(x[indexi*n:(indexi+1)*n], axis=0)
    return x_out


########################################################################################################################
# 线性拟合
def mean_spectra_new(x, n, s):
    r_in, c_in = x.shape
    r_out = int((r_in+1-n)/s)
    x_out = np.zeros([r_out, c_in])
    for indexi in range(r_out):
        x_out[indexi] = np.mean(x[indexi*s:indexi*s+n], axis=0)
    return x_out


########################################################################################################################
# 背景扣除
def correct_background(x, pixel_range, peak):
    x = x.tolist()
    back_ground_value = [x[i] for i in peak]
    background_mean = np.mean(back_ground_value)
    x_change = [xi-background_mean for xi in x[pixel_range[0]:pixel_range[1]]]
    x[pixel_range[0]:pixel_range[1]] = x_change
    x_out = np.array(x)
    return x_out


########################################################################################################################
# 自吸收矫正
def correct_self_absoption(x, position_range):
    x = x.tolist()
    mid_position = position_range[0] + int((position_range[1]-position_range[0])/2)
    x_choose = x[position_range[0]:position_range[1]]
    max_value = max(x_choose)
    x_choose1 = x[position_range[0]:mid_position]
    x_choose2 = x[mid_position:position_range[1]]
    start = x_choose1.index(max(x_choose1)) + position_range[0]
    end = x_choose2.index(max(x_choose2)) + mid_position
    x_change = [2*max_value-x for x in x[start:end]]
    x[start:end] = x_change
    x = np.array(x)
    return x


########################################################################################################################
# label repeat
def label_repeat(x, r):
    x_output = x * np.ones([r, 1])
    return x_output


########################################################################################################################
# 线性拟合
def f_1(x, a, b):
    return a * x + b


########################################################################################################################
# 除去重复
def remove_duplicate(x):  # the imput should be the list style

    x = x[:, 0].tolist()

    output = []

    for i in x:

        if i not in output:
            output.append(i)

    output = np.array(output).reshape(-1, 1)

    return output


########################################################################################################################
# 归一化同种样品的能量
def normalize_single(spectra):

    spectra_max = np.max(spectra, axis=1)

    spectra_max = spectra_max.reshape(-1, 1)

    spectra_mean = np.mean(spectra)

    spectra_output = spectra*spectra_mean/spectra_max

    return spectra_output


########################################################################################################################
# 归一化同种样品的能量
def normalize_single_1(spectra):

    wh = np.where(spectra == np.max(spectra))

    spectra_max = spectra[:, wh[1][0]]

    spectra_max = spectra_max.reshape(-1, 1)

    spectra_mean = np.mean(spectra_max)

    spectra_output = spectra * spectra_mean / spectra_max

    return spectra_output


########################################################################################################################
# 归一化同种样品的能量
def normalize_single_2(spectra):

    spectra_mean_single = np.mean(spectra, axis=1)

    spectra_mean_single = spectra_mean_single.reshape(-1, 1)

    spectra_mean = np.mean(spectra)

    spectra_output = spectra * spectra_mean / spectra_mean_single

    return spectra_output


########################################################################################################################
# 归一化样品的能量
def normalize(spectra, group_set):

    X_output = np.zeros((spectra.shape[0], spectra.shape[1]))

    temp = remove_duplicate(group_set)

    group_set = group_set[:, 0].tolist()

    for indexj in range(len(temp)):

        temp_order = [i for i, x in enumerate(group_set) if x == temp[indexj]]

        X_temp = spectra[temp_order, :]

        X_temp = normalize_single_1(X_temp)

        X_output[temp_order] = X_temp

    return X_output


########################################################################################################################
# 归一化样品的能量
def normalize1(spectra):

    spectra_mean_single = np.mean(spectra, axis=1)

    spectra_mean_single = spectra_mean_single.reshape(-1, 1)

    spectra_mean = np.mean(spectra)

    X_output = spectra * spectra_mean / spectra_mean_single

    return X_output


########################################################################################################################
# 相对标准差
def relative_standard_derivation(x_predict, x_te):  # the imput should be the list style

    output = np.std((x_predict - x_te) / x_te) #* np.sqrt(len(x_te) / (len(x_te) - 1))

    return output


########################################################################################################################
# 相对偏差
def relative_error(x_predict, x_true):  # the input should be the list style

    output = np.mean(abs(x_predict - x_true) / x_true)

    return output


########################################################################################################################
# limited of detection
def limited_of_detection(slope, rsd_calibration):

    output = 3 * np.mean(rsd_calibration) / slope

    return output


#  BPNNet
########################################################################################################################
learning_rate_max_whole = 0.3
learning_rate_min_whole = 0.2
epochs_whole = 2000000
batch_size_whole = 0
threshold = 5e-7
########################################################################################################################
# active function


def relu(x):
    return np.maximum(x, 0.0)


def relu_deriv(x):
    x[x > 0] = 1.0
    x[x <= 0] = 0.0
    return x


def tanh(x):
    return np.tanh(x)


def tanh_deriv(x):
    return 1.0 - np.tanh(x)*np.tanh(x)


def logistic(x):
    return 1/(1+np.exp(-x))


def logistic_derivative(x):
    return logistic(x)*(1-logistic(x))


def prelu(x):
    return np.maximum(x, 0.0) + 0.25*np.minimum(x, 0.0)


def prelu_deriv(x):
    x[x > 0] = 1.0
    x[x < 0] = 0.25
    return x


class BPNNet:

    def __init__(self, layers, activation = ["tanh","tanh"], batch_size="None"):

        self.layers = layers

        # active function
        self.activation = activation

        if self.activation[0] == 'logistic':

            self.activation0 = logistic

            self.activation0_deriv = logistic_derivative

        elif self.activation[0] == 'tanh':

            self.activation0 = tanh

            self.activation0_deriv = tanh_deriv

        elif self.activation[0] == 'relu':

            self.activation0 = relu

            self.activation0_deriv = relu_deriv

        elif self.activation[0] == 'prelu':

            self.activation0 = prelu

            self.activation0_deriv = prelu_deriv

        if self.activation[1] == 'logistic':

            self.activation1 = logistic

            self.activation1_deriv = logistic_derivative

        elif self.activation[1] == 'tanh':

            self.activation1 = tanh

            self.activation1_deriv = tanh_deriv

        elif self.activation[1] == 'relu':

            self.activation1 = relu

            self.activation1_deriv = relu_deriv

        elif self.activation[1] == 'prelu':

            self.activation1 = prelu

            self.activation1_deriv = prelu_deriv

        # batch_size
        if batch_size == "None":

            self.batch_size = 0

        else:

            self.batch_size = batch_size

        # random weights & weights updates
        self.weights = []

        self.weights.append((2*np.random.random((layers[0]+1,layers[1]))-1)*0.25)

        for i in range(2,len(self.layers)):

            self.weights.append((2*np.random.random((layers[i - 1],layers[i])) - 1) * 0.25)

        # random weights & weights updates

        self.updates = []

        self.updates.append(np.zeros((self.layers[0] + 1, self.layers[1])))

        for i in range(2,len(self.layers)):

            self.updates.append(np.zeros((self.layers[i - 1], self.layers[i])))

    def fit(self, X, y, learning_rate_max=0.3, learning_rate_min=0.1, epochs=2000000, error_threshold=1e-8): # learning_rate=0.2

        # self.weights 的数据更新
        self.weights[0] = (2*np.random.random((self.layers[0]+1,self.layers[1]))-1)*0.25

        for i in range(2, len(self.layers)):
            self.weights[i - 1] = (2*np.random.random((self.layers[i - 1],self.layers[i])) - 1) * 0.25

        #print(self.weights)

        # atlest_2d函数:确认X至少二位的矩阵

        X = np.atleast_2d(X)

        # 初始化矩阵全是1（行数，列数+1是为了有B这个偏向）

        temp = np.ones([X.shape[0],X.shape[1]+1])

        # 行全选，第一列到倒数第二列

        temp[:,0:-1]=X

        X = temp

        # 真实值的y数组

        y = np.array(y)

        # batch_size 的mini_batch计算

        batch_size = int(self.batch_size*X.shape[0])+1

        print(batch_size)

        # epoch 每一次循环的BP

        for k in range(epochs):

            learning_rate = learning_rate_max - (learning_rate_max-learning_rate_min)*k/epochs

            # mini_batch 随机取出规定数目的序号list
            order = random.sample(range(X.shape[0]), batch_size)

            #print(order)

            # self.updates 的数据更新
            self.updates[0] = np.zeros((self.layers[0] + 1, self.layers[1]))

            for i in range(2, len(self.layers)):
                self.updates[i-1] = np.zeros((self.layers[i - 1], self.layers[i]))

            # print(self.updates)

            # mini_batch 梯度下降抽样
            error = 0

            for j in range(batch_size):

                temp_j = order[j]

                # 根据order产生的随机数数列，循环每次一个样本

                a = [X[temp_j]]

                a.append(self.activation0(np.dot(a[0], self.weights[0])))

                a.append(self.activation1(np.dot(a[1], self.weights[1])))

                # 向前传播，得到每个节点的输出结果

                error_temp = y[temp_j] - a[-1]

                error += error_temp

            error = error/batch_size   # 最后一层错误率

            deltas = [error * self.activation1_deriv(a[-1])]

            deltas.append(deltas[-1].dot(self.weights[1].T) * self.activation0_deriv(a[1]))

            deltas.reverse()

            if abs(error) <= error_threshold:
                print(k)
                print(error)
                break
            elif k==epochs_whole-1:
                print(k)
                print(error)

            for i in range(len(self.weights)):

                layer = np.atleast_2d(a[i])

                delta = np.atleast_2d(deltas[i])

                self.updates[i] += learning_rate * layer.T.dot(delta)

            for i in range(len(self.updates)):

                self.weights[i] += self.updates[i]/batch_size

    def predict(self, x):

        # x=np.array(x)
        r1, c1 = x.shape

        aa = np.ones(shape=[r1, 1])

        for indexi in range(r1):

            temp = np.ones(c1 + 1)

            temp[0:-1] = x[indexi].T

            a = temp

            a = self.activation0(np.dot(a, self.weights[0]))

            a = self.activation1(np.dot(a, self.weights[1]))

            aa[indexi] = a

        return (aa)

    def load_weights(self, path_load):

        '''''
        worksheet = xlrd.open_workbook(path_load)
        sheet_names = worksheet.sheet_names()
        '''''
        for indexi in range(len(self.weights)):

            data = pd.read_excel(path_load, sheet_name=str(indexi),
                                 header=None, skiprows=0)

            self.weights[indexi] = np.array(data)



    def save_weights(self, path_save):
        '''''
        workbook = xlwt.Workbook(encoding='ascii')

        for indexi in range(len(self.weights)):
            cmp = self.weights[indexi]
            worksheet = workbook.add_sheet(str(indexi))
            for i in range(len(cmp)):
                for j, k in enumerate(cmp[i]):
                    worksheet.write(i, j, k)

        workbook.save(path_save)
        '''''
        with pd.ExcelWriter(path_save) as writer:
            for indexi in range(len(self.weights)):

                print(indexi)

                data = self.weights[indexi]

                data = pd.DataFrame(data)

                data.to_excel(writer, sheet_name=str(indexi), index=False, header=False)


########################################################################################################################
for choose_element in tqdm_notebook(['SiO2','Fe2O3','Al2O3','CaO','MgO','K2O','Na2O','TiO2']):
    for process in tqdm_notebook(['RawSpectrum']):
        #choose_element = 'Fe2O3' #Na2O  SiO2 Fe2O3 Al2O3 CaO MgO  Na2O TiO2
        date = '20191226'
        try_num = 1
        sub_path = os.path.join(date, choose_element+'_'+str(try_num))
        main_path = r'D:\data\china_2020\result\BPNN results'
        save_model_path = os.path.join(main_path, sub_path, process)
        #predict_path = save_model_path
        # repeat
        kk = 6
        rr = 10
        # average parameter
        mean_number = 180
        step = 180
        ########################################################################################################################
        # Input Data
        date_save = '20191226'
        save_path_file = os.path.join(main_path, date_save, choose_element+'_'+str(try_num), process)
        if not os.path.exists(save_path_file):
            os.mkdir(save_path_file)
        data_file_path = os.path.join(r'D:\data\china_2020\coeffs_dwt\coeffs_cut', choose_element)
        menu_guide_path = r'D:\data\Menu_Guide-quantitative.xlsx'
        choose_type = 'Soil_Type'
        file_list = 'Spectra_Name'
        sample_list = 'Spectra_Name'
        data_flag = pd.read_excel(menu_guide_path)
        file_names = np.array(data_flag[file_list])
        concentration = np.array(data_flag[choose_element])
        sample_type = np.array(data_flag[choose_type])
        sample_name_unique = np.array(data_flag[sample_list])
        train_test_set = np.array(data_flag['Train']).tolist()
        type_list = data_flag[choose_type].drop_duplicates(keep='first').tolist()
        flag_tr = False
        flag_te = False
        p = 150  #p可调
        ########################################################################################################################
        # train list
        #train_position = [i for i, j in enumerate(train_test_set) if j == 1]
        train_position = [1, 3, 12, 13, 21]
        # test list
        #test_position = [i for i, j in enumerate(train_test_set) if j == 0]
        test_position = [1, 3, 12, 13, 21]
        ########################################################################################################################
        state = 'rock'
        X_tr = None
        for index_i in train_position:
            temp_sample_type = type_list.index(sample_type[index_i])
            temp_sample_name = sample_name_unique[index_i]
            temp_concentration = concentration[index_i]
            for j in range(8):
                file = state+'_'+file_names[index_i]+'_'+process+'_'+str(j)+'.txt'
                read_path = os.path.join(data_file_path, file)
                if not os.path.exists(read_path):
                    print(read_path)
                    continue
                data_cache = np.loadtxt(read_path)
                #data_cache = data_cache.flatten()
                if X_tr is None:
                    X_tr = data_cache
                    y_tr = temp_concentration
                    #type_tr = temp_sample_type
                    group_tr = index_i
                else:
                    X_tr = np.row_stack((X_tr, data_cache))
                    y_tr = np.row_stack((y_tr, temp_concentration))
                    #type_tr = np.row_stack((type_tr, temp_sample_type))
                    group_tr = np.row_stack((group_tr, index_i))


        '''
        state = 'rock'
        X_tr = None
        for index_i in train_position:
            temp_sample_type = type_list.index(sample_type[index_i])
            temp_sample_name = sample_name_unique[index_i]
            temp_concentration = concentration[index_i]
            for i in range(3):
                for j in range(8):
                    file = state+'_'+file_names[index_i]+'_'+process+'_'+str(i+1)+'_'+str(j)+'.txt'
                    read_path = os.path.join(data_file_path, file)
                    if not os.path.exists(read_path):
                        print(read_path)
                        continue
                    data_cache = np.loadtxt(read_path)
                    data_cache = data_cache.flatten()
                    if X_tr is None:
                        X_tr = data_cache
                        y_tr = temp_concentration
                        type_tr = temp_sample_type
                        group_tr = index_i
                        sample_name_tr = temp_sample_name
                        intensity = data_cache
                    else:
                        X_tr = np.row_stack((X_tr, data_cache))
                        y_tr = np.row_stack((y_tr, temp_concentration))
                        type_tr = np.row_stack((type_tr, temp_sample_type))
                        sample_name_tr = np.row_stack((sample_name_tr, temp_sample_name))
                        group_tr = np.row_stack((group_tr, index_i))
        '''
        ########################################################################################################################
        # Te & Tr true concentration
        group_tr_remove_duplicate = remove_duplicate(group_tr)
        group_tr_temp = group_tr.reshape(1, -1)
        group_tr_temp = group_tr_temp[0, :].tolist()
        order_y_tr = [group_tr_temp.index(x) for x in group_tr_remove_duplicate]
        y_tr_true = y_tr[order_y_tr,:]
        ########################################################################################################################
        # standard数据归一化
        '''
        min_max_scaler1 = preprocessing.MinMaxScaler()
        X_tr = min_max_scaler1.fit_transform(X_tr)
        X_te = min_max_scaler1.transform(X_te)
        min_max_scaler2 = preprocessing.MinMaxScaler()
        y_tr = min_max_scaler2.fit_transform(y_tr)
        y_te = min_max_scaler2.transform(y_te)
        min_max_scaler3 = preprocessing.MinMaxScaler()
        group_tr_add = min_max_scaler3.fit_transform(group_tr)
        group_te_add = min_max_scaler3.transform(group_te)
        '''
        ########################################################################################################################
        # standard数据归一化for different states
        min_max_scaler1_te = preprocessing.MinMaxScaler()
        X_te = min_max_scaler1_te.fit_transform(X_tr)
        min_max_scaler2_te = preprocessing.MinMaxScaler()
        y_te = min_max_scaler2_te.fit_transform(y_tr)
        min_max_scaler3_te = preprocessing.MinMaxScaler()
        group_te_add = min_max_scaler3_te.fit_transform(group_tr)
        ########################################################################################################################
        r0, c0 = X_te.shape
        predict_temp = np.empty(shape=[X_te.shape[0], kk * rr])
        nn = BPNNet([c0, 5, 1], ['tanh','tanh'])
        t0 = time()
        for indexi in range(kk*rr):
            nn.load_weights(save_model_path + r'\weights'+str(indexi)+'.xlsx')
            temp = nn.predict(X_te)
            predict_temp[:, indexi] = temp[:, 0]
        print("done in %0.3fs" % (time() - t0))
        # 作图使用的calibration val && validation set的predicted value & errorbar
        #  predicted value
        predict_mean = np.mean(predict_temp, axis=1)
        predict_std = np.std(predict_temp, axis=1)
        num_cache = 0
        sum_cache = 0
        final_flag = True
        spectra_per_sample = 8
        pred_cache = np.zeros((spectra_per_sample))
        for i in range(len(predict_mean)):
            pred_cache[num_cache] = predict_mean[i]
            num_cache += 1
            sum_cache = sum_cache + predict_mean[i]
            if num_cache == spectra_per_sample:
                if final_flag:
                    predict_final = sum_cache/spectra_per_sample
                    true_final = y_te[i]
                    std_final = np.std(pred_cache)
                    final_flag = False
                    num_cache = 0
                    sum_cache = 0
                else:
                    predict_final = np.row_stack((predict_final, sum_cache/spectra_per_sample))
                    true_final = np.row_stack((true_final, y_te[i]))
                    std_final = np.row_stack((std_final, np.std(pred_cache)))
                    num_cache = 0
                    sum_cache = 0
        '''
        writer = pd.ExcelWriter(predict_path + r'\predict.xlsx')
        predict_results = np.column_stack((predict_mean, predict_std, y_te))
        predict_results = pd.DataFrame(predict_results)
        predict_results.to_excel(writer, header=['pred_mean', 'pred_std', 'true'], index=None)
        writer.save()
        '''
        writer = pd.ExcelWriter(save_path_file + r'\predict_final_all.xlsx')
        predict_results = np.column_stack((predict_final, true_final, std_final))
        predict_results = pd.DataFrame(predict_results)
        predict_results.to_excel(writer, header=['pred_final', 'true_final', 'std_final'], index=None)
        writer.save()


done in 1.166s



done in 1.163s



done in 1.137s



done in 1.139s



done in 1.110s



done in 1.134s



done in 1.139s



done in 1.114s




In [35]:
y_te.shape

(320, 1)

In [33]:
X_te.shape

(40, 100)

In [ ]:
# for spectra
########################################################################################################################
for choose_element in tqdm_notebook(['SiO2','Fe2O3','Al2O3','CaO','MgO','K2O','Na2O','TiO2']):
    for process in tqdm_notebook(['DenoisedSpectrum', 'SmoothedSpectrum']):
        #choose_element = 'Fe2O3' #Na2O  SiO2 Fe2O3 Al2O3 CaO MgO  Na2O TiO2
        date = '20191216'
        try_num = 1
        sub_path = os.path.join(date, choose_element)
        main_path = r'D:\data\china_2020\result\BPNN results'
        save_model_path = os.path.join(main_path, sub_path, process)
        predict_path = save_model_path
        # repeat
        kk = 6
        rr = 10
        # average parameter
        mean_number = 180
        step = 180
        ########################################################################################################################
        # Input Data
        save_path_file = save_model_path
        data_file_path = os.path.join(r'D:\data\china_2020\spectra_cut', choose_element)
        menu_guide_path = r'D:\data\Menu_Guide-quantitative.xlsx'
        choose_type = 'Soil_Type'
        file_list = 'Spectra_Name'
        sample_list = 'Spectra_Name'
        data_flag = pd.read_excel(menu_guide_path)
        file_names = np.array(data_flag[file_list])
        concentration = np.array(data_flag[choose_element])
        sample_type = np.array(data_flag[choose_type])
        sample_name_unique = np.array(data_flag[sample_list])
        train_test_set = np.array(data_flag['Train']).tolist()
        type_list = data_flag[choose_type].drop_duplicates(keep='first').tolist()
        flag_tr = False
        flag_te = False
        p = 150  #p可调
        ########################################################################################################################
        # train list
        #train_position = [i for i, j in enumerate(train_test_set) if j == 1]
        train_position = [1, 3, 12, 13, 21]
        # test list
        #test_position = [i for i, j in enumerate(train_test_set) if j == 0]
        test_position = [1, 3, 12, 13, 21]
        ########################################################################################################################
        state = 'rock'
        X_tr = None
        for index_i in train_position:
            temp_sample_type = type_list.index(sample_type[index_i])
            temp_sample_name = sample_name_unique[index_i]
            temp_concentration = concentration[index_i]
            for j in range(8):
                file = state+'_'+file_names[index_i]+'_'+process+'.txt'
                read_path = os.path.join(data_file_path, file)
                if not os.path.exists(read_path):
                    print(read_path)
                    continue
                data_cache = np.loadtxt(read_path)
                #data_cache = data_cache.flatten()
                if X_tr is None:
                    X_tr = data_cache
                    y_tr = np.ones((8)).reshape(-1, 1) * temp_concentration
                    #type_tr = temp_sample_type
                    group_tr = np.ones((8)).reshape(-1, 1) * index_i
                else:
                    X_tr = np.row_stack((X_tr, data_cache))
                    y_tr = np.row_stack((y_tr, np.ones((8)).reshape(-1, 1) * temp_concentration))
                    #type_tr = np.row_stack((type_tr, temp_sample_type))
                    group_tr = np.row_stack((group_tr, np.ones((8)).reshape(-1, 1) * index_i))